In [1]:
import gurobipy as gp
from gurobipy import GRB, quicksum
from openpyxl import load_workbook
import pandas as pd

In [9]:
# Reading input
if __name__ == '__main__':
  # Load file
 List_arcs = pd.read_excel('dt2.xlsx', sheet_name = "Arcs", index_col=False)
 List_commo = pd.read_excel('dt2.xlsx', sheet_name = "Commodities", index_col=False)

# Defining objects with corresponding features
class Arc:
 def __init__(self,origin,destination,cost, capacity):
  self.From = origin
  self.To = destination
  self.Cost = cost
  self.Capac = capacity

Arcs = []
for _ , row in List_arcs.iterrows():
  new_arc = Arc(row["From"], row["To"], row["Cost"], row["Capac"])
  Arcs.append(new_arc)

class Commodity:
 def __init__(self,origin,destination, quantity):
  self.From = origin
  self.To = destination
  self.Quant = quantity
  
Commodities=[]
for _ , row in List_commo.iterrows():
  new_commo = Commodity(row["From"], row["To"], row["Quant"])
  Commodities.append(new_commo)

class Node:
 def __init__(self, outlinks, inlinks):
  # Arcs that get in and out of the node
  self.OutLinks = outlinks# Relationship between new arc and info about in/out-links of node
  self.InLinks = inlinks
  def addOutLink(self,Node):
   self.OutLinks.appends(Node)
  def addInLink(self,Node):
   self.InLinks.append(Node)


In [18]:
# Define the nodes in the network
Nodes = [
    #1
    Node(outlinks=[3, 4], inlinks=[]),
    #2
    Node(outlinks=[3, 4], inlinks=[]),
    #3
    Node(outlinks=[3, 4], inlinks=[]),
    #4
    Node(outlinks=[5, 6, 7], inlinks=[0, 1, 2]),
    #5
    Node(outlinks=[5, 6, 7], inlinks=[0, 1, 2]),
    #6
    Node(outlinks = [],inlinks=[3,4]),
    #7
    Node(outlinks = [],inlinks=[3,4]),
    #8
    Node(outlinks = [],inlinks=[3,4])
]


In [19]:
# Initialize the objective function
objective = 0
x={}
# Loop over each arc and commodity
for m in range(len(Arcs)):
    for k in range(len(Commodities)):
        x[k, Arcs[m].From, Arcs[m].To] = 0
        # Multiply the quantity of the commodity by the cost of the arc
        # and add this to the objective function
        objective += Commodities[k].Quant * Arcs[m].Cost * x[k, Arcs[m].From, Arcs[m].To]

# Set the objective function to be minimized
#model.setObjective(objective, GRB.MINIMIZE)

In [20]:
model = gp.Model( "Model Minimize Cost")
# Loop over each arc
for m in range(len(Arcs)):
    # Create a constraint object to enforce the capacity constraint
    c = model.addConstr(
        # Sum up the flow of all commodities through the arc
        quicksum(x[k, Arcs[m].From, Arcs[m].To] for k in range(len(Commodities))),
        # Constrain the total flow to be less than or equal to the capacity of the arc
        "<=",
        Arcs[m].Capac,
        # Give the constraint a descriptive name
        name="Capacity: (%d)" % (m)
    )

In [23]:
# Create a list to store the flow continuity constraints
continuity = []

# Loop over each commodity and node
for k in range(len(Commodities)):
    for j in range(len(Nodes)):
        # If the node is the origin of the commodity, set the constraint to ensure that the total flow into the node equals the quantity of the commodity generated at the node
        if j == Commodities[k].From:
            # Create a constraint object to enforce the flow continuity constraint
            c = model.addConstr(
                # Sum up the flow of the commodity into the node from all incoming arcs
                quicksum(x[k, j, p] for p in Nodes[j].InLinks),
                # Constrain the total flow to equal the quantity of the commodity generated at the node
                "=",
                Commodities[k].Quant,
                # Give the constraint a descriptive name
                name="Flow continuity: (%d, %d)" % (k, j)
            )
            # Add the constraint to the list of flow continuity constraints
            continuity.append(c)

        # If the node is the destination of the commodity, set the constraint to ensure that the total flow out of the node equals the demand for the commodity at the node
        elif j == Commodities[k].To:
            # Create a constraint object to enforce the flow continuity constraint
            c = model.addConstr(
                # Sum up the flow of the commodity out of the node to all outgoing arcs
                quicksum(x[k, j, p] for p in Nodes[j].OutLinks),
                # Constrain the total flow to equal the demand for the commodity at the node
                "=",
                -Commodities[k].Quant,
                # Give the constraint a descriptive name
                name="Flow continuity: (%d, %d)" % (k, j)
            )
            # Add the constraint to the list of flow continuity constraints
            continuity.append(c)
                    # If the node is a transit node, set the constraint to ensure that the total flow into the node equals the total flow out of the node
        # else:
        #     if 0 in x:
        #         # Key 0 is defined, so we can safely access the value
        #         value = x[0]
        #     else:
        #         # Key 0 is not defined, so we can't access the value
        #         value = None
        #     # Create a constraint object to enforce the flow continuity constraint
        #     c = model.addConstr(
        #         # Sum up the flow of the commodity into the node from all incoming arcs
        #         quicksum(x[k, j, p] for p in Nodes[j].InLinks),
        #         # Constrain the total flow into the node to equal the total flow out of the node
        #         "=",
        #         # Sum up the flow of the commodity out of the node to all outgoing arcs
        #         quicksum(x[k, j, p] for p in Nodes[j].OutLinks),
        #         # Give the constraint a descriptive name
        #         name="Flow continuity: (%d, %d)" % (k, j)
        #     )
        #     # Add the constraint to the list of flow continuity constraints
        #     continuity.append(c)


KeyError: (4, 3, 0)